# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [0]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [0]:
# Clone the entire repo.
!git clone -l -s git://github.com/acrucetta/disaster_response_pipeline.git cloned-repo
%cd cloned-repo
!ls

In [0]:
%pwd
%cd data

In [0]:
# load messages dataset
messages = pd.read_csv('/content/cloned-repo/data/disaster_messages.csv')
messages.head()

In [0]:
# load categories dataset
categories = pd.read_csv('/content/cloned-repo/data/disaster_categories.csv')
categories.head()

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [0]:
# merge datasets
df = pd.merge(messages,categories,how = 'inner', left_on = "id", right_on = "id")
df.head()

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [0]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(";",expand = True)
categories.head()

In [0]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.str.slice(stop= -2)
print(category_colnames)

In [0]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [0]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str.slice(start = -1)
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
    
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [0]:
df.head()

In [0]:
# drop the original categories column from `df`
df = df.drop(columns = ['categories'])

df.head()

In [0]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df,categories], axis=1)
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [0]:
# check number of duplicates
df.duplicated().value_counts()
df.info()

In [0]:
# drop duplicates
df = df.drop_duplicates()

In [0]:
# check number of duplicates
df.duplicated().value_counts()

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [0]:
engine = create_engine('sqlite:///disaster_response.db')
df.to_sql('messages_cat', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [0]:
df = pd.read_sql_table("messages_cat",engine)

In [0]:
df.message[2]

In [0]:
df

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### Importing libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [0]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
import pickle

import nltk
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

from nltk.corpus import stopwords

In [0]:
df = pd.read_sql_table('messages_cat', engine)
X = df['message']
y = df[category_colnames_clean]

In [0]:
category_colnames_clean = category_colnames.drop(labels = [0,35,34,27,18, 28])
category_colnames_clean

In [0]:
# Dropping abstract categories from Y to ease analyis
#y = y.drop(columns= ["related","other_infrastructure","other_weather","other_aid","direct_report","weather_related"])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
stop_words = set(stopwords.words("english"))

### Writing a tokenization function to process our text data

In [0]:
def tokenize(text):
    """
    Normalize, tokenize and stems texts.
    
    Input:
    text: string. Sentence containing a message.
    
    Output:
    stemmed_tokens: list of strings. A list of strings containing normalized and stemmed tokens.
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text) 
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Pipeline #1 - K Nearest Neighbors [Default]
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [0]:
def model_pipeline():
    pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(KNeighborsClassifier()))
                ])
    return pipeline

#### Train pipeline
- Split data into train and test sets
- Train pipeline

In [0]:
model_v1 = model_pipeline()
model_v1.fit(X_train, y_train)

In [0]:
y_pred = model_v1.predict(X_test)

### Pipeline #2 - KNN With Grid Search CV
Use grid search to find better parameters. 

In [0]:
def model_pipeline_with_cv():
    pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())])),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
    
    parameters = {
        'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_neighbors': (10,15)
    }
         
    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = 4, verbose = 2)
    
    return cv

#### Train pipeline
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [0]:
model_v2 = model_pipeline_with_cv()
model_v2.fit(X_train, y_train)

In [0]:
y_pred_v2 = model_v2.predict(X_test)

### Pipeline v3 - Random Forest Classifier with Grid Search CV

In [0]:
class StartVerbExtractor(BaseEstimator, TransformerMixin):
    def start_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) != 0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
        return 0


    def fit(self, X, y=None):
        return self
    

    def transform(self, X):
        X_tag = pd.Series(X).apply(self.start_verb)
        return pd.DataFrame(X_tag)

In [0]:
def build_model_v3():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
    ])
    return pipeline

#### Train pipeline

In [0]:
build_model_v3().get_params()

In [0]:
model_v3 = build_model_v3()
model_v3.fit(X_train, y_train)

In [0]:
y_pred_v3 = model_v3.predict(X_test)

### Testing pipelines

In [0]:
print('KNN Without Grid Search CV')
#print(classification_report(y_test, y_pred, target_names = category_colnames))

print('KNN With Grid Search CV')
#print(classification_report(y_test, y_pred_v2, target_names = category_colnames))

print('Random Forest with Starting Verb Extractor')
print(classification_report(y_test, y_pred_v3, target_names = category_colnames_clean))

### Exporting the model as a pickle file

In [0]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(model_v3, 'classifier.pkl') 

In [0]:
df = pd.read_sql_table('messages_cat', engine)
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
y = y.drop(columns=["related", "other_infrastructure", "other_weather", "other_aid", "direct_report", "weather_related"])
category_names = y.columns.tolist()


In [0]:
y

## Visualizations for web application

In [0]:
from plotly.graph_objs import Bar
import json
import plotly
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import plotly.graph_objects as go

In [0]:
    # extract data needed for visuals
    genre_counts = df.groupby('genre').count()['message']
    genre_names = list(genre_counts.index)
    category_df = df.drop(columns=['id', 'message', 'genre'])
    category_sums = pd.DataFrame(np.sum(category_df), columns=["count"]).sort_values(['count'], ascending=False)

    # create visuals
    graphs = [
        {
            'data': [
                Bar(
                    x=genre_counts,
                    y=genre_names,
                    orientation = 'h'
                )
            ],
            'layout': {
                'title': 'Distribution of Message Genres',
                'xaxis': {
                    'title': "Count"
                },
                'yaxis': {
                    'title': "Genre"
                }
            }
        },
        {
            'data': [
                    Bar(
                        y= category_sums.index,
                        x=category_sums['count'],
                        orientation = 'h'
                    )
                ],
            'layout': {
                'title': "Distribution of Message Types",
                "xaxis": {
                    'title':'Count'
                },
                'yaxis': {
                    'title':'Message Type'
                }
            }
        }
    ]

    # encode plotly graphs in JSON
    ids = ["graph-{}".format(i) for i, _ in enumerate(graphs)]
    graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)